<a href="https://colab.research.google.com/github/hochschule-pforzheim/project-st23-team-f23/blob/main/1_Cleansing_Join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Laden der wichtigsten Komponenten**

> Zuerst muss Pandas (unter der Abkürzung pd) und der Datensatz der csv-Datei ([hotel_orig.csv](https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv)) als 'hotel_orig' in das Jupiter Notebook geladen und als Dataframe über die Funktion '`read.csv()`' gespeichert werden. Die Spalte 'reservation_status_date' wird mit dem 'parse_dates' Befehl als Dateityp Dateum übergeben.


In [ ]:
import pandas as pd
hotel_orig = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv', parse_dates=['reservation_status_date'])

> Um einen ersten Eindruck über den Datensatz sowie den Aufbau des Dataframes zu bekommen, wird das Dataframe ausgegeben. Um mehr als die nur die ersten und letzten 5 Zeilen angezeigt zu bekommen, setzen wir die minimale Anzahl an gezeigten Reihen mit `pd.set_option('display.min_rows', 50)` auf 50 und mit `pd.set_option('display.max_columns', 500)` können wir alle Variablen im DataFrame ausgeben lassen.


In [ ]:
# set options to show more rows and columns for exploration
pd.set_option('display.min_rows', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
hotel_orig

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,NaN,NaN,0,Transient,107.00,0,0,Check-Out,2015-07-03
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,0.0,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303.0,NaN,0,Transient,103.00,0,1,Check-Out,2015-07-03
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,82.00,0,1,Canceled,2015-05-06
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,15.0,NaN,0,Transient,105.50,0,0,Canceled,2015-04-22


## **Erste relevante Strukturanpassungen als Vorbereitung für spätere Analysen**







> Da das Datum der Buchung in 3 verschiedenen Spalten (arrival_date_year, arrival_date_month, arrival_date_day_of_month) im Datensatz hinterlegt ist, werden diese Spalten zur einfacheren Auswertung vereint. Mit dem Befehl des Panda Paketes 'to_datetime' wird das anschließend, in Klammern stehende, Argument zu einem Datum transformiert. Es werden die drei einzelen Spalten zu einem String transformiert (`'astype(str)'`) und anschließend mit einem - voneinander getrennt, hintereinander weg in eine neue Spalte namens 'arrival_date' im Dataframe hinterlegt. Diese hat das Format yyyy-mm-dd.






In [ ]:
# melt various date columns to one readable date-column
hotel_orig['arrival_date'] = pd.to_datetime(hotel_orig.arrival_date_year.astype(str) + '-' + hotel_orig.arrival_date_month.astype(str) + '-' + hotel_orig.arrival_date_day_of_month.astype(str))

> Um die neue Spalte 'arrival_date' nicht als letzte Spalte des Dataframes anzuhängen, ändern wir über die Funktion '`insert()`' die Position der neuen Spalte zum Index 3 der x-Achse (der Achse, auf der die Spalten liegen). Dafür wird die Spalte, die ursprünglich am Ende eingefügt war, verwendet und am ursprünglichen Ort entfernt.

In [ ]:
# change order of columns (put new arrival_date-column to the beginning)
hotel_orig.insert(3, 'arrival_date', hotel_orig.pop('arrival_date'))



> Als ID in dem Dataframe wird nun die Spalte 'night_index' erstellt, welche einfach die Reihen von oben nach unten durchzählt und so einen eindeutigen identifier bereitstellt. Dieser Idex wird noch an Position 1 gesetzt.



In [ ]:
hotel_orig['night_index'] = range(len(hotel_orig))
hotel_orig.loc[:, 'night_index'] = range(len(hotel_orig))

In [ ]:
hotel_orig.insert(0, 'night_index', hotel_orig.pop('night_index'))



> Für spätere Analysen scheint eine noch nicht vorhandene Zeile im Datensatz wichtig, die Summe der Nächte, für jede Buchung. Hierfür werden die gebuchten Nächte in der Woche und die gebuchten Nächte am Wochenende aufsummiert und in einer neuen Spalte namens 'stay_nights_sum' abgespeichert.



In [ ]:
hotel_orig['stay_nights_sum'] = hotel_orig['stays_in_week_nights'] + hotel_orig['stays_in_weekend_nights'] 

> Mit der Funktion '`value_counts()`' kann im nächsten Schritt bestimmt werden, wie häufig ein gleicher Wert in einer bestimmten Spalte angegeben wurde. So haben die meisten buchenden Personen (27643) für 2 Nächte gebucht. Für sehr lange Zeiträume, beispielsweise für 34, 57 oder 43 Nächte, haben jeweils nur eine Person gebucht.


In [ ]:
hotel_orig.stay_nights_sum.value_counts() #.describe auch angucken

2     27643
3     27076
1     21020
4     17383
7      8655
5      7784
6      3857
8      1161
10     1139
14      916
9       841
0       715
11      396
12      223
13      142
15       75
21       71
16       40
25       37
18       35
28       35
19       22
17       20
29       14
20       14
22       14
30       13
23        8
24        6
26        6
27        5
35        5
42        4
33        3
56        2
34        1
57        1
49        1
48        1
69        1
38        1
45        1
60        1
46        1
43        1
Name: stay_nights_sum, dtype: int64

Im nächsten Schritt wird der Datensatz um die Spalte 'departure_date' erweitert. Hierfür ist die Funktion timedelta des Paketes datetime notwendig. Eine lambda Funktion fügt der 'arrival_date'-Zeit den Wert der Spalte "stay_nights_sum" Tage hinzu und gibt das Ergebnis zurück.

In [ ]:
from datetime import timedelta

hotel_orig['departure_date'] = hotel_orig.apply(lambda row: row['arrival_date'] + timedelta(days=row['stay_nights_sum']), axis=1)

In [ ]:
hotel_orig.departure_date.describe()

<ipython-input-10-b1b17073224a>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  hotel_orig.departure_date.describe()


count                  119390
unique                    806
top       2015-12-08 00:00:00
freq                      452
first     2015-07-01 00:00:00
last      2017-09-14 00:00:00
Name: departure_date, dtype: object

Die beiden Spalten 'departure_date' und 'stay_nights_sum' werden an passendere Positionen im Datensatz verschoben.

In [ ]:
hotel_orig.insert(5, 'departure_date', hotel_orig.pop('departure_date'))
hotel_orig.insert(12, 'stay_nights_sum', hotel_orig.pop('stay_nights_sum'))


> Das Dataframe kann nun durch Eingabe des gespeicherten Variablennamen und Ausführen der Codezelle mit den vorgenommenen Einstellungen und Änderungen angezeigt werden.



In [ ]:
# show datafrane
hotel_orig

,night_index,hotel,is_canceled,lead_time,arrival_date,departure_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,stay_nights_sum,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,0,342,2015-07-01,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,1,Resort Hotel,0,737,2015-07-01,2015-07-01,2015,July,27,1,0,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,2,Resort Hotel,0,7,2015-07-01,2015-07-02,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,3,Resort Hotel,0,13,2015-07-01,2015-07-02,2015,July,27,1,0,1,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,4,Resort Hotel,0,14,2015-07-01,2015-07-03,2015,July,27,1,0,2,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
5,5,Resort Hotel,0,14,2015-07-01,2015-07-03,2015,July,27,1,0,2,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
6,6,Resort Hotel,0,0,2015-07-01,2015-07-03,2015,July,27,1,0,2,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,NaN,NaN,0,Transient,107.00,0,0,Check-Out,2015-07-03
7,7,Resort Hotel,0,9,2015-07-01,2015-07-03,2015,July,27,1,0,2,2,2,0.0,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303.0,NaN,0,Transient,103.00,0,1,Check-Out,2015-07-03
8,8,Resort Hotel,1,85,2015-07-01,2015-07-04,2015,July,27,1,0,3,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,82.00,0,1,Canceled,2015-05-06
9,9,Resort Hotel,1,75,2015-07-01,2015-07-04,2015,July,27,1,0,3,3,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,15.0,NaN,0,Transient,105.50,0,0,Canceled,2015-04-22




---



# Prüfung der Datenqualität



> Im nächsten Schritt kann nun der vorliegende Satz auf fehlende Werte überprüft werden. Hierzu eignet sich die Funktion `isna()`. `isna()` ermittelt alle fehlende Werte pro Spalte und Zeile. Wird `isna()` mit `sum()` verkettet, so wird die Gesamtanzahl fehlender Werte pro Spalte ausgegeben.



In [ ]:
# print sum of missing values per row
hotel_orig.isna().sum()

night_index                            0
hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date                           0
departure_date                         0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
stay_nights_sum                        0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_ty



> Im Hotel-Datensatz sind die meisten Zellen mit Werten gefüllt, lediglich vier Spalten enthalten 'Missing Values' (children, country, agent und company). In der 'children'-Spalte fehlen 4 Angaben, unter 'country' fehlen bei 488 Datenzeilen die jeweiligen Werte, bei 'agent' sind es sogar 16340 und unter 'company' wurden in 112593 Zeilen keine Angaben vorgenommen.

> Durch Teilen der Summe aller fehlender Werte pro Spalte durch die Gesamtlände des Dataframes kann das Verhältnis fehlender Werte zu Gesamtwerten ermittelt werden. Wird dieses Ergebnis dann mit `apply('{0:.4%}'.format)` bearbeitet, wird es als Prozentwert ausgegeben.





In [ ]:
# print percentage of missing values per column
(hotel_orig.isna().sum()/len(hotel_orig)).apply('{0:.4%}'.format)

night_index                        0.0000%
hotel                              0.0000%
is_canceled                        0.0000%
lead_time                          0.0000%
arrival_date                       0.0000%
departure_date                     0.0000%
arrival_date_year                  0.0000%
arrival_date_month                 0.0000%
arrival_date_week_number           0.0000%
arrival_date_day_of_month          0.0000%
stays_in_weekend_nights            0.0000%
stays_in_week_nights               0.0000%
stay_nights_sum                    0.0000%
adults                             0.0000%
children                           0.0034%
babies                             0.0000%
meal                               0.0000%
country                            0.4087%
market_segment                     0.0000%
distribution_channel               0.0000%
is_repeated_guest                  0.0000%
previous_cancellations             0.0000%
previous_bookings_not_canceled     0.0000%
reserved_ro


> Die heraus stechenden Spalten sind, wie auch oben schon beschrieben, 'company' mit ~94.3% und 'agent' mit ~13.7%. Für die Spalten 'country' und 'children' wurde ein Prozentsatz an fehlenden Werten von ~0.4% bzw. 0.0034% errechnet.

> Die Spalte 'company' beschreibt dabei Geschäftskunden bzw. Kunden, die über ihr Unternehmen gebucht haben. Trotz der vielen Missing Values, ist diese Spalte für das Projekt allerdings zweitrangig, weshalb die Werte zu vernachlässigen sind.

> In der Spalte 'agent' konnten ebenfalls, im Vergleich zu anderen Spalten, viele Missing Values verzeichnet werden. In dieser Spalte wurden Buchungen festgehalten, die über ein Reisebüro getätigt wurden. Da auch diese Information für das Projekt zweitrangig ist, sind die fehlenden Werte zu vernachlässigen.

> Dadurch, dass lediglich vier Spalten Missing Values enthalten und diese Spalten für die geplanten Anaylsen eher zweitrangig sind, lässt sich darauf schließen, dass der Datensatz für das Projekt gut geeignet ist.

- Agent sind Personen, die über ein Reisebüro gebucht haben 

- company und Agent für unsere Analysen eher zweitrangig, deswegen egal dass so viele Missing Values

